<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/llm_makes_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
import os
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
# Set base parameters

MODEL_NAME = "Qwen/Qwen2.5-7B-Instruct"

LOAD_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/Scrapes"
LOAD_FILE = os.path.join(LOAD_PATH, "bipolar_scrape.json")

SAVE_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/JSON"
os.makedirs(SAVE_PATH, exist_ok=True)
SAVE_FILE = os.path.join(SAVE_PATH, "bipolar.json")

In [ ]:
# Load LLM

"""
Focus should be on instruction-following models from Hugging Face
With free licence (Apache)
Qwen seems to have been trained on producing JSON formats
...allows for many tokens as input
...parameters are good balance between small and big
Should also check Llama offerings?
"""

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto")

In [ ]:
# Test

## Should also carry out test prompt of transforming recommendations

text = "Should someone with a diagnosis of bipolar who is taking lithium be referred to secondary care if they are mildly irritable?"

inputs = tokenizer(text, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs, max_new_tokens=200)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

Should someone with a diagnosis of bipolar who is taking lithium be referred to secondary care if they are mildly irritable? The answer is yes, because:
A: Lithium levels may be elevated
B: They are at risk of developing mania
C: Lithium toxicity is more likely in irritable patients
D: Lithium levels may be decreased

The correct answer is A: Lithium levels may be elevated.

Explanation:
Irritability can be a side effect of lithium, and it's important for healthcare providers to monitor this symptom. If a patient with bipolar disorder who is on lithium therapy is experiencing mild irritability, it could indicate that their lithium levels are elevated. Elevated lithium levels can lead to toxicity, which has various adverse effects including gastrointestinal symptoms, tremor, confusion, and in severe cases, life-threatening complications.

While the other options (B, C, and D) might also be relevant considerations, they do not directly address the primary concern of irritability as a pot

In [4]:
# Load JSON of raw recommendations

def load_json():
    try:
        with open(LOAD_FILE, "r", encoding="utf-8") as raw_recommendations:
            return json.load(raw_recommendations)
    except FileNotFoundError:
        raise FileNotFoundError(f'JSON file not found: {LOAD_FILE}')

raw_recommendations = load_json()

print(type(raw_recommendations))
print(len(raw_recommendations))
print(raw_recommendations[0])

<class 'list'>
136
{'heading_1': '1.1 Care for adults, children and young people across all phases of bipolar disorder', 'sub_heading_1': 'Treatment and support for specific populations', 'sub_heading_2': None, 'r_number': '1.1.1', 'r_text': 'Ensure that older people with bipolar disorder are offered the same range of treatments and services as younger people with bipolar disorder. '}


In [ ]:


def extract_one_recommendation():

        for item in raw_rec:

        heading_1 = []
        sub_heading_1 = []
        sub_heading_2 = []
        r_number = []
        r_text = []

        heading_1.append(item["heading_1"])
        sub_heading_1.append(item["sub_heading_1"])
        if item.get("sub_heading_2") != None:
            sub_heading_2.append(item["sub_heading_2"])

In [6]:
# Prompt LLM to translate raw recommendations into reasoning format

## Check OpenAI prompt engineering guide

def construct_prompt():

    """
    Given one recommendation entry {}, creates the prompt to extract one normalised JSON item
    """

    heading_context = " > ".join(heading_1, sub_heading_1, sub_heading_2)

    return f"""
    You are extracting structured information from a NICE guideline recommendation.

    RULES:
    - output must be valid JSON only (no markdown)
    - do not invent clinical information, thresholds or populations; use only what is present
    - 'action' is a concrete imperative
    - if there is more than one action, retain all
    - 'conditionality' is determined by clauses that begin 'if...' or 'where...'
    - 'prohibitions' are 'do not', 'must not' and 'should not'
    - record urgency as 'True' if the text includes 'urgent', 'urgently', 'immediate' or 'immediately'
    - you MUST use 'null' if the information is not explicit in the recommendation or heading

    CONTEXT: {heading_context}

    RECOMMENDATION NUMBER: {r_number}
    RECOMMENDATION TEXT:M {r_text}

    Produce JSON with exactly these keys:
    - action
    - scope
    - population
    - conditionality
    - prohibitions
    - urgency
    - original_recommendation_number
    - original_recommendation_text
    """

In [5]:
def run_llm_on_entity(tokenizer, model, entity):

    """
    Call the model on a single prompt using the prompt function
    Return model response
    """

    prompt = construct_prompt(entity)

    inputs = tokenizer(prompt,
                       return_tensors="pt").to(model.device)

    outputs = model.generate(**inputs,
                             max_new_tokens=500,
                             temperature=0)

    response = tokenizer.batch_decode(outputs[:, inputs.shape[1]:],
                                      skip_special_tokens=True)

    return response[0]

In [ ]:
def convert_output_to_true_json():

	"""
    Takes output from run_llm_on_entity
    Turns it into a true JSON dictionary
    """

    llm_response = llm_response.strip()

    start = text.find("{")
    end = text.rfind("}")

    json_object = llm_response[start:end + 1].strip()

    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        raise
    except ValidationError as e:
        print(f"Validation error: {e}")
        raise
    except Exception as e:
        print(f"Unexpected error: {e}")
        raise

    return True





In [ ]:
def validate_json(json_object, required_keys: list):

        missing_keys = [k for k in required_keys if k not in json_object]
        if missing_keys:
            raise ValueError(f"Missing required keys: {missing_keys}")

        extra_keys = [k for k in json_object.keys() if k not in required_keys]
            if extra_keys:
                raise ValueError(f"Unexpected extra keys: {extra_keys}")

        for key, value in json_object.items():
            if isinstance(value, (str))
            continue
        raise TypeError(f"Key '{key}' has unsupported type {type(value)}")

    return json_object

In [ ]:




    json.dump(DATA_VARIABLE, open(SAVE_FILE.replace(".txt", ".json"), "w", encoding="utf-8"), ensure_ascii=False, indent=2)
